In [13]:
import numpy as np
import math
import residuals as r

# from function_base import mycorrcoef

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

In [23]:
# I am defining the weighted standard deviation according to this definition:
# https://en.wikipedia.org/wiki/Weighted_arithmetic_mean#Weighted_sample_variance
# I am using the inverse square of the error arrays to define the weights

factor10 = math.sqrt((25*60)/10.0) 
factor80 = math.sqrt((25*60)/80.0) 



def weighted_stddev(values, errors):
    '''
    Return the weighted average and standard deviation.

    '''
    weights = 1. / ((errors)**2)
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (math.sqrt(variance))

def magnitude(x):
    var = float(x)
    return int(math.floor(math.log10(var)))

def jitter_ratio(resids_file, condition_lower, condition_upper, titlearg):
    '''
    Return the ratio of sigma_dump over sigma_all.
    
    Added the correlation coefficient too, so we can easily compare here.

    '''
    
    #Define sigma_all (or std_all) as entire observation period
    x=r.read_residuals(filename=resids_file)

    condition = (x.bary_TOA > condition_lower) & (x.bary_TOA < condition_upper)
    Resid_all = x.prefit_sec[condition]
    Errors_all = x.uncertainty[condition]
    std_all = weighted_stddev(Resid_all, Errors_all)
    
    ##============
    
    #Define sigma_dump (or std_all) as individual dumps within observation period
    
    #This isolates the numbers for the different dump times
    rounded = np.round(np.array([x.bary_TOA[condition] - 0.00005]), 4)
    dumptimes=[]
    for i in rounded.tolist()[0]: 
        if i not in dumptimes:
            dumptimes.append(i)

    #This calculates a list of sigma_dump values and averages those
    str_dump_devs=[]
    for i in dumptimes:
        timeindex = (rounded == i)[0]
        Resid = Resid_all[timeindex]
        Errors = Errors_all[timeindex]
        single_std_dump = weighted_stddev(Resid, Errors)
        str_dump_devs.append(single_std_dump)
    
    #to convert str_dump_devs out of scientific notation
    dump_devs=[]
    for i in str_dump_devs:
        dump_devs.append('{0:.20f}'.format(i))
        
    #prints the magnitude of each sigma_dump in a list and finds average magnitude
    magarray=[]
    for i in dump_devs:
        magarray.append(magnitude(i))
    averagemag = round(np.mean(magarray))
    
    arg=averagemag+1.5
    makemask = np.ma.masked_where(np.array(magarray) > arg, np.array(magarray))
    
    std_dump=np.mean(str_dump_devs)
    masked_std_dump = np.mean(np.ma.masked_where(np.ma.getmask(makemask), str_dump_devs))
    
    #Setup jitter ratio
    ratio = std_all / std_dump
    masked_ratio = std_all / masked_std_dump
    
    ##============
    
    #Print out the correlation coefficient also, to allow for easy reference
    
    #Creating the matrix of the data and transposing it to correct format
    data = []
    data_errors = []
    for i in dumptimes[1:]: #the first set of resids needs to be skipped in J1713, but not sure for all of them?
        timeindex = (rounded == i)[0]
        Resids = Resid_all[timeindex]
        Errors = Errors_all[timeindex]
        data.append(Resids)
        data_errors.append(Errors)

    data = map(list, zip(*data)) #transposing the matrix, unnecessary if rowvar=0
#     data = np.array(data)
#     weights = 1. / (np.array(data_errors)**2)
        
    #Using the Weighted Correlation Coefficient from wikipedia
    #https://en.wikipedia.org/wiki/Pearson_product-moment_correlation_coefficient#Weighted_correlation_coefficient
    # I added weights to the numpy correlation, using their pre-weighted covariance function
    # I don't need to transpose data if rowvar is =0, this is easier for constructing the matrix
    # of error weights
        
    #Correlating! 
    corr = np.corrcoef(data).tolist()
#     weighted_corr = mycorrcoef(data, rowvar=0, aweights=weights)

    unique_corrs = []
    #Averaging the correlation
    for i in range(len(corr))[:-1]:
        unique_corrs.append(corr[i][i+1:]) #because numpy array, double index necessay
    unique_corrs  = [val for sublist in unique_corrs for val in sublist]
    final_coefficient = np.average(unique_corrs)
    
#     unique_weighted_corrs = []
#     #Averaging the correlation
#     for i in range(len(weighted_corr))[:-1]:
#         unique_corrs.append(weighted_corr[i][i+1:]) #because numpy array, double index necessay
#     unique_weighted_corrs  = [val for sublist in unique_weighted_corrs for val in sublist]
#     final_weighted_coefficient = np.average(unique_weighted_corrs)
    
    print titlearg
    print "sigma_all = " + str(std_all)
    print "average sigma_dump = " + str(std_dump)
    print "jitter ratio = " + str(ratio) 
    print "masked jitter ratio = " + str(masked_ratio) 
    print "correlation coefficient = " + str(final_coefficient)
#     print "weighted correlation coefficient = " + str(final_weighted_coefficient)

In [24]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56380_80F8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 80 second subintervals')

J1713+0747, MJD 56380, 80 second subintervals
sigma_all = 2.44415585816e-07
average sigma_dump = 2.49624000145e-08
jitter ratio = 9.79134961678
masked jitter ratio = 9.79134961678
correlation coefficient = 0.975310619846


In [25]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56380_NTF8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 10 second subintervals')

J1713+0747, MJD 56380, 10 second subintervals
sigma_all = 5.82671845721e-07
average sigma_dump = 7.10113781891e-08
jitter ratio = 8.20533075938
masked jitter ratio = 8.20533075938
correlation coefficient = 0.965601343627


In [26]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56201_80F8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 80 second subintervals')

J1713+0747, MJD 56201, 80 second subintervals
sigma_all = 1.73501235065e-07
average sigma_dump = 5.08948679605e-08
jitter ratio = 3.40901238215
masked jitter ratio = 3.40901238215
correlation coefficient = 0.893879830465


In [27]:
jitter_ratio("/nimrod1/eschwab/residuals/J1713+0747_resid_56201_NTF8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 10 second subintervals')

J1713+0747, MJD 56201, 10 second subintervals
sigma_all = 5.43224438005e-07
average sigma_dump = 8.13611454109e-08
jitter ratio = 6.67670588045
masked jitter ratio = 6.67670588045
correlation coefficient = 0.916150377776


In [28]:
jitter_ratio("/nimrod1/eschwab/residuals/B1937+21_resid_56465_80F8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 80 second subintervals')

B1937+21, MJD 56465, 80 second subintervals
sigma_all = 4.68059107539e-08
average sigma_dump = 4.44210286622e-08
jitter ratio = 1.05368813293
masked jitter ratio = 1.05368813293
correlation coefficient = 0.0816201709277


In [29]:
jitter_ratio("/nimrod1/eschwab/residuals/B1937+21_resid_56465_NTF8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 10 second subintervals')

B1937+21, MJD 56465, 10 second subintervals
sigma_all = 7.05138954322e-08
average sigma_dump = 4.92378535069e-08
jitter ratio = 1.43210742163
masked jitter ratio = 1.43210742163
correlation coefficient = 0.771068051713


In [30]:
jitter_ratio("/nimrod1/eschwab/residuals/J1022+1001_resid_57208_80F8.tmp",
            57208.778, 57208.79, 'J1022+1001, MJD 57208, 80 second subintervals')

J1022+1001, MJD 57208, 80 second subintervals
sigma_all = 4.6866049668e-05
average sigma_dump = 0.000296708047933
jitter ratio = 0.157953415806
masked jitter ratio = 3.33695161071
correlation coefficient = -0.0908035595199


In [31]:
jitter_ratio("/nimrod1/eschwab/residuals/J1022+1001_resid_57208_NTF8.tmp",
            57208.778, 57208.79, 'J1022+1001, MJD 57208, 10 second subintervals')

J1022+1001, MJD 57208, 10 second subintervals
sigma_all = 0.000104350827388
average sigma_dump = 0.000345928859937
jitter ratio = 0.301654008881
masked jitter ratio = 2.1828169481
correlation coefficient = -0.270679064499


In [32]:
jitter_ratio("/nimrod1/eschwab/residuals/J1022+1001_resid_57331_80F8.tmp",
            57331.443, 57331.4595, 'J1022+1001, MJD 57331, 10 second subintervals')

J1022+1001, MJD 57331, 10 second subintervals
sigma_all = 1.65569569649e-06
average sigma_dump = 1.43649632382e-06
jitter ratio = 1.15259306204
masked jitter ratio = 1.15259306204
correlation coefficient = 0.131051206325


In [33]:
jitter_ratio("/nimrod1/eschwab/residuals/J1022+1001_resid_57331_NTF8.tmp",
            57331.443, 57331.4595, 'J1022+1001, MJD 57331, 10 second subintervals')

J1022+1001, MJD 57331, 10 second subintervals
sigma_all = 1.99889485593e-05
average sigma_dump = 1.28371737411e-05
jitter ratio = 1.55711443675
masked jitter ratio = 2.07273433488
correlation coefficient = 0.0634166037353


In [34]:
jitter_ratio("/nimrod1/eschwab/residuals/J2317+1439_resid_57273_80F8.tmp",
            57273.196, 57273.217, 'J2317+1439, MJD 57273, 80 second subintervals')

J2317+1439, MJD 57273, 80 second subintervals
sigma_all = 8.63069774192e-05
average sigma_dump = 7.9531672163e-05
jitter ratio = 1.08519002646
masked jitter ratio = 1.08519002646
correlation coefficient = 0.393373284044


In [35]:
jitter_ratio("/nimrod1/eschwab/residuals/J2317+1439_resid_57273_NTF8.tmp",
            57273.196, 57273.217, 'J2317+1439, MJD 57273, 10 second subintervals')

J2317+1439, MJD 57273, 10 second subintervals
sigma_all = 0.000136392509784
average sigma_dump = 0.000136221578037
jitter ratio = 1.00125480668
masked jitter ratio = 1.00125480668
correlation coefficient = -0.0165572422927


In [36]:
jitter_ratio("/nimrod1/eschwab/residuals/J2234+0944_resid_57069_80F8.tmp",
            57069.7, 57069.72, 'J2234+0944, MJD 57069, 80 second subintervals')

J2234+0944, MJD 57069, 80 second subintervals
sigma_all = 3.09374810381e-07
average sigma_dump = 2.54509387228e-07
jitter ratio = 1.21557327905
masked jitter ratio = 1.21557327905
correlation coefficient = 0.244021824527


In [37]:
jitter_ratio("/nimrod1/eschwab/residuals/J2234+0944_resid_57069_NTF8.tmp",
            57069.7, 57069.72, 'J2234+0944, MJD 57069, 10 second subintervals')

J2234+0944, MJD 57069, 10 second subintervals
sigma_all = 7.79080622293e-07
average sigma_dump = 6.23925410863e-07
jitter ratio = 1.24867589736
masked jitter ratio = 1.24867589736
correlation coefficient = 0.181644203051


In [ ]:
# writing out the correlation code

#first i need to isolate the arrays. There should be 8 in most, but some still have 16.
#(rereduce J1713 and B1937 with updated jitterpipe)

#I'll be isolating the arrays of residuals, they will all be residuals of the same frequency band
#going forward in time

#starting with J1713, even though I'm going to make it divisible by 14 frequency bands.
#usually this will be 8 or 9


#Defining arrays
x=r.read_residuals(filename="/nimrod1/eschwab/residuals/J1713+0747_resid_56380_80F8.tmp")

date = 56380
condition=(x.bary_TOA > 56380.35) & (x.bary_TOA < 56380.367)

TOA=x.bary_TOA[condition]
Resid=x.prefit_sec[condition]
Freq=x.bary_freq[condition]
Error=x.uncertainty[condition]

#Creating a matrix of arrays in the correct order
# Rows of residuals by frequency
# Columns by TOA
rounded = np.round(np.array([x.bary_TOA[condition] - 0.00005]), 4)
dumptimes=[]
for i in rounded.tolist()[0]: 
    if i not in dumptimes:
        dumptimes.append(i)

#Creating the data and transposing it to correct format
data = []
for i in dumptimes[1:]:
    timeindex = (rounded == i)[0]
    Resids = Resid[timeindex]
    data.append(list(Resids))
realdata = map(list, zip(*data))

#Correlating!
corr = np.corrcoef(realdata).tolist()

unique_corrs = []
#Averaging the correlation
for i in range(len(corr))[:-1]:
    unique_corrs.append(corr[i][i+1:]) #because numpy array, double index necessay
unique_corrs  = [val for sublist in unique_corrs for val in sublist]
np.average(unique_corrs)